# 字符

##### 概念
* Unicode ：码位，是 0~1,114,111 的数字，在 Unicode 标准中以 4~6 个十六进制数字表示，并且加前缀 "U+"，如 A 的码位是 U+0041，约 10% 的有效码位有对应的字符，字符范围为人类可读的文本
* ASCII、utf-8、utf-16 等：编码，是码位和字节序列间转换使用的算法
* b'caf\xc\xa9'：字节序列，供以机器读取的文本
##### 各个字节值可能由以下三种方式显示
* 可打印的 ASCII 范围内的字节 (从空格到 ~ )，使用 ASCII 字符本身
* 制表符、换行符、回车符和 \ 对应的字节，使用转义序列 \t、\n、\r 和 \\
* 其他字节的值，使用十六进制转义序列，如 \x00 是空字节

In [1]:
# 'café' 字符串有 4 个 Unicode 字符
s  = 'café'
len(s)

4

In [2]:
# 使用 UTF-8 把 str 对象编码成 bytes 对象
b = s.encode('utf-8')
b

b'caf\xc3\xa9'

In [3]:
# 字节序列有5个字节，其中 'é' 在 UTF-8 编码中为 2 个字节
len(b)

5

In [4]:
# 使用 UTF-8 把 bytes 对象解码成 str 对象
b.decode('utf-8')

'café'

In [5]:
# 各个元素是 range(256) 内的整数
b[0]

99

In [6]:
# bytes 对象的切片仍是 bytes 对象，即时只是一个字节的切片
b[:1]

b'c'

二进制序列有个类方法是 str 没有的，名为 fromhex，它的作用是解析十六进制数字对(数字对之间的空格是可选的)，构建二进制序列

In [7]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

In [8]:
# 使用数组中的原始数据初始化 bytes 对象
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
otcets = bytes(numbers)
otcets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

##### 结构体和内存视图
struct 模块提供了一些函数，把打包的字节序列转换成不同类型字段组成的元组，还有一些函数用于执行反向转换，把元组转换成打包的字节序列。  
struct 模块能处理 bytes、bytearray 和 memoryview 对象

In [11]:
# 使用 memoryview 和 struct 查看一个 GIF 图像的首部
import struct
fmt = '<3s3sHH' # < 是小字节序，3s3s 是两个 3 字节序列，HH 是两个 16 位二进制整数
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())

header = img[:10]
bytes(header) 

b'GIF89a\xfa\x00\x02\x01'

In [12]:
# 拆包 memoryview 对象，得到一个元组，包含类型、版本、宽度和高度
struct.unpack(fmt, header)

(b'GIF', b'89a', 250, 258)

In [13]:
# 删除引用，释放内存
del header
del img

# 4.4 处理编码问题

##### 判断字符编码
统一字符编码侦测包 Chardet

# 4.6 为了正确比较而规范化 Unicode 字符串

In [ ]:
# U+0301 是 COMBINING ACUTE ACCENT，加在 'e' 后得到 'é'，在 Unicode 中，他们被称为“标准等价物”，应用程序应将他们视作相同的字符。但 Python 看到的是不同的码位序列，因此判定二者不同
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [ ]:
len(s1), len(s2) 

(4, 5)

In [ ]:
s1 == s2

False

这个问题的解决方案是使用 unicodedata.normalize 函数提供的 Unicode 规范化。这个函数的第一个参数位这 4 个字符串中的一个：
* 'NFC'
* 'NFD'
* 'NFKC'
* 'NFKD'

In [ ]:
# NFC (Normalization Form C) 使用最少的码位构成等价的字符串
# NFD 把组合字符解成基字符和单独的组合字符
# 这两种规范化方式都能让比较行为符合预期
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
len(s1), len(s2)

(4, 5)

In [ ]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [ ]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [ ]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [ ]:
normalize('NFD', s1) == normalize('NFD', s2)

True

In [ ]:
# 在另外两个规范化形式 (NFKC 和 NFKD) 的首字母缩写词中，K 表示 compatibility (兼容性)
# 他们对 "兼容字符" 有英雄。虽然 Unicode 的目标是为各个字符提供规范的码位，但为了兼容现有标准，有些字符会出现多次
# 如 "μ" 这个字符，码位是 U+03BC，但同时也有 U+00B5 以便与 latin1 互相转换。所以它是一个兼容字符
# 在 NFKC 和 NFKD 模式中，各个兼容字符会被替换成一个或多个兼容分解字符，即使格式有所损失
# 例："½" 二分之一通过兼容分解后得到的是三个字符序列 "1⁄2"
from unicodedata import normalize, name
half = '½'
normalize('NFKC', half)

'1⁄2'

In [ ]:
four_squared = '4²'
normalize('NFKC', four_squared)

'42'

因此 NFKD 与 NFKC 有可能会损失或曲解信息，尽量不要在存储时使用，但它相当适合在查询时使用

大小写折叠
将所有文本变为小写再做其他转换。这个功能由 str.casefold() 方法支持
注意它与 str.lower() 不同

## 4.6.2 规范化文本匹配实用函数

In [ ]:
from unicodedata import normalize

# Unicode 标准等价物比较
def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

# 不区分大小写比较
def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold())

## 4.6.3 极端规范化 去掉变音符号

In [ ]:
import unicodedata
import string

def shave_marks(txt):
    """去掉全部变音符号"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt
                    if not unicodedata.combining(c)) # 过滤掉所有组合记号
    return unicodedata.normalize('NFC', shaved)

In [ ]:
s1 = 'café'
shave_marks(s1)

'cafe'

In [ ]:
# str.translate() 进行映射
single_map = str.maketrans("""01234""","""56789""")
multi_map = str.maketrans({
    'q': 'as',
    'w': 'df'
})
multi_map.update(single_map)
'a1b2c3q4w5'.translate(multi_map)

'a6b7c8as9df5'

# 4.7 Unicode 文本排序

In [ ]:
# 使用 pyuca.Collator.sort_key 方法
# PyUCA 是 Unicode 排序算法的纯 Python 实现
import pyuca
coll = pyuca.Collator()
sorted = sorted(txt, key = coll.sort_key)

# 4.8 Unicode 数据库与正则表达式 re 对其的支持

Unicode 提供了一个完整的数据库，不仅包括码位与字符名称之间的映射，还有各个字符的元数据一级字符之间的关系。  
例如：字符是否可以打印、是不是字母、是不是数字、是不是其他数值服号

In [ ]:
# tag::NUMERICS_DEMO[]
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print(f'U+{ord(char):04x}',                       # U+0000 格式的码位
          char.center(6),                             # 在长度为 6 的字符串中居中显示字符
          're_dig' if re_digit.match(char) else '-',  # 如果字符匹配正则表达式 r'\d'，显示 re_dig
          'isdig' if char.isdigit() else '-',         # 如果 char.isdiget() 返回 True，显示 isdig
          'isnum' if char.isnumeric() else '-',       # 如果 char.isnumeric() 返回 True，显示 isnum
          f'{unicodedata.numeric(char):5.2f}',        # 使用长度为 5、小数点后保留 2 位的浮点数显示数值
          unicodedata.name(char),                     # Unicode 标准中字符的名称
          sep='\t')
# end::NUMERICS_DEMO[]

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


正则表达式并不能完全匹配 isdigit 方法判断为数字的字符。re 模块对 Unicode 的支持并不充分。
而 regex 模块可以提供更好的 Unicode 支持

在正则表达式中 \d \w 等模式只能匹配 ASCII字符  
但如果是字符串模式，就能匹配 ASCII 之外的 Unicode 数字或字母。
* 字节序列只能由字节序列正则表达式搜索，字节序列即编码后的 Unicode ，即 str.encode('utf-8') 等 
* 字符串模式 r'\d+' 能匹配泰米尔数字和 ASCII 数字
* 字节序列模式 rb'\d+' 只能匹配 ASCII 字节中的数字
* 字符串模式 r'\w+' 能匹配字幕、上标、泰米尔数字和 ASCII 数字
* 字节序列模式 rb'\w+' 只能匹配 ASCII 字节中的字母和数字

字符串正则表达式有个 re.ASCII 标志，它让表达式只能匹配 ASCII 字符